# Cleanning Shark CSV

Importing usefull libraries

In [39]:
import pandas as pd
import numpy as np
import re
import SRC.functions as fc

In [40]:
df = pd.read_csv('../../Data project - Resources/attacks.csv', encoding='latin-1')

## Analizing data

### 1. Understand what type of data contains the .csv *

In [41]:
#How many row and collumns does it has
print(f"The df has {df.shape} (rows/colums)")
print(f"Coulms: {fc.columns(df)}")
print(f"Rows: {fc.rows(df)}")

The df has (25723, 24) (rows/colums)
Coulms: 24
Rows: 25723


In [42]:
# What kind of info can I find?
print(f"The colums are the followin: {list(df.columns)}")

The colums are the followin: ['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']


### 2. Deleting usless info

Analize the rows and columns in the dataframe that do not add value 

--> 2.1 Delete duplicates

In [43]:
#delete al duplicates
df = df.drop_duplicates()

In [44]:
# Analyze again
print(f"The df has {df.shape} (rows/colums)")
print(f"Coulms: {fc.columns(df)}")
print(f"Rows: {fc.rows(df)}")

The df has (6312, 24) (rows/colums)
Coulms: 24
Rows: 6312


--> 2.2 Delete columns with 95% of more missing info

In [45]:
# How many colums has missing data? --> All of them 🤦🏻‍♀️
df.isnull().sum().count()

24

In [46]:
# Which column has the most missing data?
print(f"Number of columns with missing info: {df.isnull().sum().count()}")
print(f"Percentage of columns with missing info: {fc.per_c_miss_info(df)} %")
df.isnull().sum().sort_values(ascending=False)

Number of columns with missing info: 24
Percentage of columns with missing info: 100.0 %


Unnamed: 22               6311
Unnamed: 23               6310
Time                      3364
Species                   2848
Age                       2841
Sex                        575
Activity                   554
Location                   550
Fatal (Y/N)                549
Area                       465
Name                       220
Country                     60
Injury                      38
Investigator or Source      27
Type                        14
Year                        12
href formula                11
pdf                         10
href                        10
Case Number.1               10
Case Number.2               10
Date                        10
original order               3
Case Number                  2
dtype: int64

In [47]:
# Analize columns that has more than 95% info missing
print(fc.col_miss_95(df))

['Unnamed: 22', 'Unnamed: 23']


In [48]:
# Those columns are not adding value so I'll delete them
df = df.drop(fc.col_miss_95(df), axis=1)

In [49]:
# Validating that the columns with more than 95% null has been deleted
print(df.isnull().sum().sort_values(ascending=False))
print(f"Columns: {fc.columns(df)}")

Time                      3364
Species                   2848
Age                       2841
Sex                        575
Activity                   554
Location                   550
Fatal (Y/N)                549
Area                       465
Name                       220
Country                     60
Injury                      38
Investigator or Source      27
Type                        14
Year                        12
href formula                11
Case Number.2               10
Date                        10
pdf                         10
href                        10
Case Number.1               10
original order               3
Case Number                  2
dtype: int64
Columns: 22


--> 2.3 Delete rows with 95% of more missing info

In [50]:
# How many rows has missing data? --> All of them 🤦🏻‍♀️
df.T.isnull().sum().count()

6312

In [51]:
# Analize rows that has more than 95% info missing
print(f"Rows: {fc.rows(df)}")
fc.row_miss_95(df)

Rows: 6312


[6309, 8702, 25722]

In [52]:
# Those rows are not adding value so I'll delete them
df = df.drop(fc.row_miss_95(df))

In [53]:
# Validating that the rows with more than 95% null has been deleted
print(f"Rows: {fc.rows(df)}")
fc.row_miss_95(df)

Rows: 6309


[]

### 3. Cleaning data to validate the hipothesis

Deleting the not relevant info for the hipothesis validation:
 * The highest percentage of shark attacks to surfers happened in California

In [54]:
# Review what info contains every column and slect what is usefull for the investigation and what is not

In [55]:
df.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
dtype: object

In [56]:
df_date = df[['Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order']]
df_date.head(10)

,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
5,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.b,2018.06.03.b,6298.0
6,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.a,2018.06.03.a,6297.0
7,"K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.27,2018.05.27,6296.0
8,"K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.b,2018.05.26.b,6295.0
9,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.a,2018.05.26.a,6294.0


In [57]:
#Deleting columns that are not adding value (because I don't know their meanning)
df = df.drop(columns=['Investigator or Source', 'pdf', 'href formula', 'href', 
                 'Case Number.1', 'Case Number.2', 'original order'])

In [58]:
# After claning the usless columns, I'll check if there is some duplicate rows
df = df.drop_duplicates()
fc.rows(df)

6303

In [59]:
# Deleting usless rows
df= df.drop(6302)
df.tail()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN
6301,ND.0001,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN


In [60]:
print(df.isnull().sum().sort_values(ascending=False))

Time           3354
Species        2838
Age            2831
Sex             565
Activity        544
Location        540
Fatal (Y/N)     539
Area            455
Name            210
Country          50
Injury           28
Type              4
Year              2
Case Number       1
Date              0
dtype: int64


In [61]:
# Time and Species and Sex will not de usefull for this investigation and has a los of null values
df = df.drop(columns=['Time', 'Species ', 'Sex ', 'Age'])

In [62]:
fc.print_shape(df)

The df has (6302, 11) (rows/colums)
Rows: 6302
Coulms: 11
    Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Injury', 'Fatal (Y/N)'],
      dtype='object')


In [63]:
# Investigatinf the rest of the columns

In [64]:
df['Case Number'].value_counts().sort_index()
# Case Number seem to be useless
df = df.drop(columns=['Case Number'])

In [65]:
df['Year'].value_counts().sort_index()
# Year seem to be useless
df = df.drop(columns=['Year'])

In [66]:
df['Type'].value_counts().sort_index()

Boat             137
Boating          203
Boatomg            1
Invalid          547
Provoked         574
Questionable       2
Sea Disaster     239
Unprovoked      4595
Name: Type, dtype: int64

In [67]:
df['Area'].value_counts().sort_values(ascending=False)
# This si useful to split by area

Florida                  1037
New South Wales           486
Queensland                311
Hawaii                    298
California                290
                         ... 
Sumatra                     1
30 nm from Singapore        1
Oslo Fjord                  1
Milne Bay  Province         1
Ho Ha Wan Marine Park       1
Name: Area, Length: 825, dtype: int64

In [68]:
df['Location'].value_counts().sort_values()
# Location seem to be useless because there is too much differenet locations
df = df.drop(columns=['Location'])

In [69]:
df['Activity'].value_counts().sort_values(ascending=False)
# This si useful to split by activity

Surfing                                            971
Swimming                                           869
Fishing                                            431
Spearfishing                                       333
Bathing                                            162
                                                  ... 
Net fishing                                          1
Bitten after dhow shipwrecked                        1
Dived into sea from launch & bitten immediately      1
Adrift on refugee raft                               1
Fishing from Surfboard                               1
Name: Activity, Length: 1532, dtype: int64

In [70]:
df['Name'].value_counts().sort_index()
# Name seem to be useless
df = df.drop(columns=['Name'])

In [71]:
df['Injury'].value_counts().sort_index()
# Injury seem to be useless
df = df.drop(columns=['Injury'])

In [72]:
df['Fatal (Y/N)'].value_counts().sort_index()
# 'Fatal (Y/N)' seem to be useless
df = df.drop(columns=['Fatal (Y/N)'])

In [73]:
fc.print_shape(df)

The df has (6302, 5) (rows/colums)
Rows: 6302
Coulms: 5
    Index(['Date', 'Type', 'Country', 'Area', 'Activity'], dtype='object')


In [74]:
# Delete duplicates again
df_clean = df.drop_duplicates()

In [75]:
fc.print_shape(df_clean)
df_clean

The df has (6166, 5) (rows/colums)
Rows: 6166
Coulms: 5
    Index(['Date', 'Type', 'Country', 'Area', 'Activity'], dtype='object')


,Date,Type,Country,Area,Activity
0,25-Jun-2018,Boating,USA,California,Paddling
1,18-Jun-2018,Unprovoked,USA,Georgia,Standing
2,09-Jun-2018,Invalid,USA,Hawaii,Surfing
3,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Surfing
4,04-Jun-2018,Provoked,MEXICO,Colima,Free diving
...,...,...,...,...,...
6297,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Diving
6298,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Pearl diving
6299,1900-1905,Unprovoked,USA,North Carolina,Swimming
6300,1883-1889,Unprovoked,PANAMA,NaN,NaN


### 4. Exporting clean file to .csv

In [76]:
df_clean.to_csv("../../Data project - Resources/attacks_clean.csv")